In [1]:
import os
import networkx as nx
from random import sample,choices,uniform, randint,shuffle
import math
from DSSCircuit_Interface import *
from Outage_Eval import *
# from Outage_Eval2 import *
import numpy
import pickle

In [2]:
# Read the .gml graph and load it into notebook, get the node list
FolderName=os.path.dirname(os.path.realpath("__file__"))
G_original = nx.read_gml(""+ FolderName+r"\8500busEx.gml")
nodes=list(G_original.nodes())
edges=list(G_original.edges())

In [3]:
DSSfile=r""+ FolderName+ "\Master.dss"    

In [4]:
OutageData=[]

In [5]:
### Base Case- No contingency , Normal operation of the network at maximum loading ####
d_obj= DSS(DSSfile) #create a circuit object
d_obj.compile_ckt_dss() #compiling the circuit
d_obj.dssText.command = "Set Maxiterations=200" 
d_obj.dssText.command = "Set maxcontroliter=200"
d_obj.AllowForms = False 
d_obj.dssText.command = "Set controlmode=OFF"
d_obj.dssSolution.Solve()
# d_obj.solve_snapshot_dss(1.0) #solving snapshot power flow      

In [6]:
# Getting Load Variation as a 24 hour profile
d_obj.dssText.command='redirect Loadshapes.dss' # load multiplication factor
d_obj.dssCircuit.Loadshapes.Name = 'loadshape_multload' # Input Load shape to get mult factor
LoadMult= list(d_obj.dssCircuit.Loadshapes.Pmult) # use the 24 hour load shape with 1 hour resolution

In [7]:
#Ensuring all lines are in closed state
for e in G_original.edges(data=True):
    name = e[2]['device'] + '.' +e[2]['label']
    d_obj.dssCircuit.SetActiveElement(name)
    d_obj.dssCktElement.Close(1,0)
    d_obj.dssSolution.Solve()

In [9]:
# Parameters
Ttot=24
Nsc = 15 # No.of scenarios (for each node with subgraphs)
max_rad = int(nx.diameter(G_original)/2) # maximum radius to create a subgraph
#max_rad= 1
init_node = sample(nodes, k=Nsc) #random sampling of nodes with replacement for the large no. of scenarios

In [8]:
len(nodes)

132

In [10]:
count = 0
out_perc = [0, 0.2, 0, 0.4]
for i in out_perc:
    for nd in init_node: # for each node with center create a subgraph
        for rad in range(0, max_rad+1): # the radius is varied within range
            Gsub=nx.ego_graph(G_original, nd , radius=rad, undirected=False) #form subgraph around selected node with given radius
            sub_edges=list(Gsub.edges()) # list of subgraph edges
            #out_perc = uniform(0, 0.2) # percentage of edge failure within subgraph
            N_out=math.ceil(len(sub_edges)*i) # number of edge outages within subgraph
            out_edges=sample(sub_edges,k=N_out) # random sampling without replacement # random failure of edges within subgraph
            Outedge_dicts=[{'edge':(u,v),'device':G_original[u][v].get("device"),'label':G_original[u][v].get("label")} for (u,v) in out_edges]
            if len(out_edges)==0:
                time_out=0
            else:
                time_out=randint(1,Ttot)          
            (Bus_Pserved,Bus_PDemand,Bus_Voltages,BranchFlow,OutageType) = Outage_evaluate(DSSfile,Outedge_dicts,LoadMult[time_out-1],G_original)
            Scenario_Info={'Scenario':count,'Edges failed':Outedge_dicts,'Bus PDemand':Bus_PDemand,'Bus Voltages':Bus_Voltages,'Bus Pserved':Bus_Pserved, 'BranchFlow':BranchFlow, 'OutageType':OutageType}
            OutageData.append(Scenario_Info)
            for e in G_original.edges(data=True):
                name = e[2]['device'] + '.' +e[2]['label']
                d_obj.dssCircuit.SetActiveElement(name)
                d_obj.dssCktElement.Close(1,0)
                d_obj.dssSolution.Solve()
            count=count+1
            print(count)
            

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
shuffle(OutageData)

In [12]:
f = open('dataset.pkl', 'wb')
pickle.dump(OutageData,f)
f.close()

In [ ]:
'799': array([0.89309199, 1.0425644 , 0.91923736]),

In [14]:
OutageData[10]

{'Scenario': 51,
 'Edges failed': [],
 'Bus PDemand': {'sourcebus': 0,
  '800': 0,
  '802': 0,
  'mid806': 41.58,
  '806': 0,
  '808': 0,
  'mid810': 12.096,
  '810': 0,
  '812': 0,
  '814': 0,
  '814r': 0,
  '850': 0,
  '816': 0,
  '818': 0,
  'mid824': 3.7800000000000002,
  '824': 0,
  'mid820': 25.704,
  '820': 0,
  'mid822': 102.06,
  '822': 0,
  'mid826': 30.240000000000002,
  '826': 0,
  'mid828': 3.024,
  '828': 0,
  'mid830': 5.292,
  '830': 34.019999999999996,
  '854': 0,
  '832': 0,
  'mid858': 11.34,
  '858': 0,
  'mid834': 24.192,
  '834': 0,
  '842': 0,
  '836': 0,
  'mid840': 30.240000000000002,
  '840': 20.412,
  'mid844': 6.804,
  '844': 306.18,
  'mid846': 34.019999999999996,
  '846': 0,
  'mid848': 17.388,
  '848': 45.36,
  'mid856': 3.024,
  '856': 0,
  '852': 0,
  '852r': 0,
  '888': 0,
  'mid864': 1.512,
  '864': 0,
  'mid860': 110.376,
  '860': 45.36,
  'mid836': 61.992000000000004,
  '862': 0,
  'mid838': 21.168,
  '838': 0,
  '890': 340.2},
 'Bus Voltages': {'so

In [16]:
len(OutageData)

320

In [16]:
n = 0
for i in range(len(OutageData)):
    if OutageData[i]['OutageType'] == 'Yes':
        n = n+1
print(n)

147


In [25]:
n = 0
for i in range(len(OutageData)):
    if OutageData[i]['OutageType'] == '3-Phase Outage':
        n = n+1
        break
        
print(i)

10


In [18]:
0 = 270
1 = 50
2 = 107
3 = 53

SyntaxError: cannot assign to literal (870210027.py, line 1)

In [19]:
for i in range(len(OutageData)):
    if OutageData[i]['OutageType'] == '3-Phase Outage':
        break
    
print(i)

10


In [11]:
for i in range(len(OutageData)):
    print(OutageData[i]['OutageType'])

No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage
No Outage


len(OutageData)

In [21]:
d_obj.dssCircuit.SetActiveElement('Line.'+ 'l32')
d_obj.dssCktElement.Close(1,0)
d_obj.dssSolution.Solve()
S = d_obj.dssCktElement.Powers
S

(42.68025599381531,
 67.06481445562734,
 5.844544782007688e-08,
 -0.10265941407169674,
 82.82261947309914,
 -6.962819209809826,
 -42.62084980821647,
 -67.0660159680576,
 -4.309527053436568e-09,
 0.05992939639769892,
 -82.76311183065705,
 6.9582024590644656)